# **Install necessary packages and library**

In [ ]:
# !pip install -q --upgrade ipython==7.9.0
# !pip install -q --upgrade ipykernel==5.3.4

In [ ]:
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install datasets
!pip install paramiko

In [ ]:
!nvidia-smi

In [ ]:


import GPUtil
GPUtil.getAvailable()

import torch
use_cuda = torch.cuda.is_available()

if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

if torch.cuda.is_available():
  dev = "cuda:2"
else:
  dev = "cpu"
device = torch.device(dev)

# **unicode analyzer custom function**

In [ ]:
# import re

# def removeTheUnicodeFromText(batch):
#   chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\।\"u\\\u[0-9A-Fa-f]+]'
#   batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "

#   return batch




# **Read data from server**

# **Read csv data**

In [ ]:
# df= pd.read_csv('/content/drive/MyDrive/Bangla_STT/map_path_txt_phn.csv')
# df.head()

In [ ]:
# df['speech'][0]


# **Map wav file with text file**

In [ ]:
import paramiko, re
from tqdm import tqdm
import sys, os, stat
import torchaudio
from paramiko import AutoAddPolicy
import pandas as pd

def get_file_paths(client, path, pattern):
    stdin, stdout, stderr = client.exec_command(f"find {path} -name *{pattern}  -type f -print0 | xargs -0 ls -t")
    output = stdout.read().decode().strip().split('\n')
    return output



client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(AutoAddPolicy())
client.connect(hostname="SET YOUR HOST NAME", username="SET YOUR USER NAME", password="SET YOUR PASSWORD")
sftp_client = client.open_sftp()


list_txts = []
chunk_size = 10**3
list_of_wavs = get_file_paths(client, "PUT THE API URL", ".wav")[:20]

for path in tqdm(list_of_wavs):
    id = path.split('/')[-1].split('.')[0]
    folder = path.split('/')[4]
    text_path = f"PUT THE BASE PATH/{folder}/PUT THE ADDITIONAL PATH{id}.txt"
    text_file = sftp_client.open(text_path)
    #text_file = str(text_file)
    text = ""
    for line in text_file:
        text = line
    list_txts.append(text)
    # print(text_path)


df_new = pd.DataFrame(list(zip(list_of_wavs, list_txts)),
               columns = ['audio_path', 'label'])

df_new.head()

In [ ]:
df_new['label'] = df_new['label'].str.replace(r"_1|_2","",regex = True)
df_new.head()

In [ ]:
df_new['audio_path'][0]

In [ ]:
df_new.head()

In [ ]:
len(df_new)

In [ ]:
train_data = df_new[:8]
dev_data  = df_new[8:14]
test_Data = df_new[14:]

# **Base phoneme**

In [ ]:
vocab_dict =  {'a': 0, 'ã': 1, 'b': 2, 'bʰ': 3, 'c': 4, 'cʰ': 5, 'd': 6, 'dʰ': 7, 'd̪': 8, 'd̪ʰ': 9, 'e': 10, 'ẽ': 11, 
               'g': 12, 'gʰ': 13, 'h': 14, 'i': 15, 'ĩ': 16, 'i̯': 17, 'k': 18, 'kʰ': 19, 'l': 20, 'm': 21, 'n': 22, 
               'o': 23, 'õ': 24, 'o̯': 25, 'p': 26, 'pʰ': 27, 'r': 28, 's': 29, 't': 30, 'tʰ': 31, 't̪': 32, 't̪ʰ': 33, 'u': 34,
               'ũ': 35, 'u̯': 36, 'æ': 37, 'æ̃': 38, 'ŋ': 39, 'ɔ': 40, 'ɔ̃': 41, 'ɟ': 42, 'ɟʰ': 43, 'ɽ': 44, 'ɽʰ': 45, 'ʃ': 46, 
               'ʲ': 47, 'ʷ': 48, ' ': 49}

In [ ]:
print(len(vocab_dict))

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

print(vocab_dict)

In [ ]:
from datasets import Dataset, load_metric

train_data = Dataset.from_pandas(train_data)
dev_data = Dataset.from_pandas(dev_data)


In [ ]:
train_data

In [ ]:
#/content/drive/MyDrive/Bangla_STT/base_phoneme.json


import json
with open('/home/elias/base_phoneme.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:

print(vocab_dict)


In [ ]:
print(len(vocab_dict))

In [ ]:
from transformers import Wav2Vec2CTCTokenizer
import pandas as pd
tokenizer = Wav2Vec2CTCTokenizer("/home/elias/base_phoneme.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
processor.save_pretrained("/content/drive/MyDrive/Bangla_STT/")

In [ ]:
train_data

In [ ]:
import torchaudio

def speech_file_to_array_fn(batch):
    sftp_client = client.open_sftp()
    remote_file = sftp_client.open(batch["audio_path"])

    speech_array, sampling_rate = torchaudio.load( remote_file )
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["label"]
    return batch

train_data = train_data.map(speech_file_to_array_fn, remove_columns = train_data.column_names)
dev_data = dev_data.map(speech_file_to_array_fn, remove_columns=dev_data.column_names)



In [ ]:
import librosa
import numpy as np

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48000, 16_000)
    batch["sampling_rate"] = 16_000
    return batch

train_data = train_data.map(resample, num_proc=4)
dev_data = dev_data.map(resample, num_proc=4)



In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train_data))

ipd.Audio(data=np.asarray(train_data[rand_int]["speech"]), autoplay=True, rate=6000)

In [ ]:
rand_int = random.randint(0, len(train_data)-1)

print("Target text:", train_data[rand_int]["target_text"])
print("Input array shape:", np.asarray(train_data[rand_int]["speech"]).shape)
print("Sampling rate:", train_data[rand_int]["sampling_rate"])

In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

train_data = train_data.map(prepare_dataset, remove_columns=train_data.column_names, batch_size=32, num_proc=4, batched=True)
dev_data = dev_data.map(prepare_dataset, remove_columns=dev_data.column_names, batch_size=32, num_proc=4, batched=True)

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    
    mlflow.lod("wer",wer)
    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    ctc_zero_infinity=True,
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

In [ ]:
model.freeze_feature_extractor()

# **mlflow setup**

In [ ]:
pip install mlflow

In [ ]:
from huggingface_hub import notebook_login
import mlflow
import mlflow.pytorch
mlflow.set_experiment('LearnML-Demo')

In [ ]:
os.environ["MLFLOW_EXPERIMENT_NAME"] = "0"
os.environ["MLFLOW_FLATTEN_PARAMS"] = "1"

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  hub_model_id = "/home/eliasm/wav2vec2-mlflow", 
  output_dir= "/home/elias/",

  group_by_length=True,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=4e-4,
  warmup_steps=int(0.1*3600),
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=  model,
    data_collator=  data_collator,
    args=  training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=dev_data,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

In [ ]:
#mlflow.end_run()

In [ ]:
#trainer.push_to_hub()

In [ ]:
mlflow.set_tracking_uri("SET THE URL") 
experiment_id = mlflow.get_experiment_by_name("wav2vec2")
if experiment_id is None:
    experiment_id = mlflow.create_experiment("wav2vec2")
else:
    experiment_id = experiment_id.experiment_id

In [ ]:
with mlflow.start_run(experiment_id= experiment_id):
  trainer.train()

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = ""
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

In [ ]:
#!mlflow ui